# Objective function `the best place for the company to grow`

##### Objective function ->

X (var independient): Val_by_place Each place will be assigned with a value in each component (9 x 1)

Constants: 
     - Total personal (9 x 1)
     - Valoration of the contribution to grow by role (9 x 1)
     - Rel_personal_Val_place represent the relation between condition and personal (9 x 9)

Any places will be valuated with the next equation:

             `Constants`                      `Independient Variable`

F   =    (   Personal   *   Val_to_grow   *   Rel_personal_Val_place  )    *    Val_by_place

In [9]:
import numpy as np
import json
from pymongo import MongoClient
import pandas as pd

import sys
sys.path.insert(0, 'src/')

from funobjetivo import valor


In [ ]:
# Example of using function objective
Val_by_place={"other Design companies":0.1,
         "kindergarden":0.2,
         "success startup":0.5,
         "Starbucks":1,
         "Airport_or_train_station":2,
         "place_to party":2,
         "basqueball court":1,
         "no_2km_old_company":0.8,
         "vegan_restaurant":0.5}

valor(**Val_by_place)

## Other design companies

In [ ]:
# Connect and get the data --> Categories
url_mongo="mongodb://localhost:27017"
client = MongoClient(url_mongo)
db = client.get_database("companies")
result=list(db.companies.find({},{"category_code":1,"_id":0}))
categories=[e['category_code'] for e in result]

In [ ]:
# Count the number of design comanies and include Web 
categorias= pd.DataFrame(categories)
categorias=categorias.rename(columns={0:"categoria"})
len(categorias.categoria.value_counts())
# -->  41 categories
len(categorias.categoria[categorias.categoria=="design"]) 
# --> just 4 companies!!
len(categorias.categoria[categorias.categoria=="web"]) 
# --> 3787.. is web companies equivalent to design 
#     companies in this context?

In [ ]:
# Getting the ubication of the offices of design and web companies
query={"$or":[{"category_code":"design"},{"category_code":"web"}]}
offices=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
offices=pd.DataFrame(offices)
offices=offices.explode("offices")
offices

In [ ]:
# Extracting Lat an Long
def json_column(row):
    of=row.offices
    if type(of)==float or of["latitude"] is None:
        return (row['name'],"theres is no data ",None, None)
    return (row['name'],"success",of["latitude"] ,of["longitude"])

offices_coor_name = offices.apply(json_column, axis=1, result_type='expand')
offices_coor_name# = pd.concat([offices.name,offices_coor])

In [ ]:
offices_coor_name=offices_coor_name.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
offices_coor_name=offices_coor_name[offices_coor_name['State_coor']=="success"] 
# --> 2311 companies with coordenates

In [ ]:
# Setting the format for MongoDB index
def coor_jgeo(row):
     return {'type':'Point','coordinates':[row['longitud'],row['latitude']]}

offices_coor_name['design_coor']=offices_coor_name.apply(coor_jgeo,axis=1)

In [ ]:
offices_coor_name.to_json('output/design.json',orient="records")

## Success startup

In [ ]:
# Query for getting the rounds data
query={"funding_rounds.0.raised_amount":{"$gt":0}}

fund=list(db.companies.find(query,{"name":1,"funding_rounds":1,"_id":0}))
fund=pd.DataFrame(fund)
fund=fund.explode("funding_rounds") # several rounds...

In [ ]:
# Creating columns or Raised amount and currency
def extract_round(var):
    return var['raised_amount']

def extract_currency(var):
    return var['raised_currency_code']

fund['val_round']=fund.funding_rounds.apply(extract_round)
fund['val_currency']=fund.funding_rounds.apply(extract_currency)

In [ ]:
# Currencies of the Rounds
fund.groupby(['val_currency']).count()

In [ ]:
# Creating the rounds in USD--> (average currency exchange last 5Y)
def calculate_usd(row):
    if row['val_round'] is None or row['val_currency'] is None:
        return 0
    
    currecy_equi={'CAD':0.8,'EUR':1.2,'GBP':1.5,'JPY':0.01,'SEK':0.15,'USD':1}
    val=row['val_round'] * currecy_equi[row['val_currency']]

    return val

fund['Val_round_USD']=fund.apply(calculate_usd,axis=1)

Here we have the startups that have funds greater than 1 million dollars

In [ ]:
fund=fund[['name','Val_round_USD']].groupby(['name']).sum()
fund=fund.loc[fund.Val_round_USD>1e6]
# --> 3686 companies with more than a millon raised

##### Now find the Lat Long

In [ ]:
query={"funding_rounds.0.raised_amount":{"$gt":0}}

ss_point=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
ss_point=pd.DataFrame(ss_point)
ss_point=ss_point.explode('offices')

In [ ]:
ss_coor=ss_point.apply(json_column, axis=1, result_type='expand')
ss_coor=ss_coor.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
ss_coor.State_coor.value_counts()

In [ ]:
ss_coor=ss_coor.loc[ss_coor.State_coor=="success"]
# --> 3815 points of succesful companies
ss_coor.columns

In [ ]:
# Setting the format for MongoDB index and export json
ss_coor['success_coor']=ss_coor.apply(coor_jgeo,axis=1)
ss_coor.to_json('output/success.json',orient="records")

## Old Companies

In [ ]:
# Query for getting the founded year
query={"founded_year":{"$lt":2010}}

old=list(db.companies.find(query,{"name":1,"offices":1,"_id":0}))
old=pd.DataFrame(old)
old=old.explode("offices") # several offices
old

In [ ]:
old=old.apply(json_column, axis=1, result_type='expand')
old=old.rename(columns={0:'name',1:'State_coor',2:'latitude',3:'longitud'} )
old.State_coor.value_counts()

In [ ]:
old=old.loc[old.State_coor=="success"]
# --> 9039 points of old companies (<2010!!)
old.columns

In [ ]:
# Setting the format for MongoDB index and export it
old['success_coor']=old.apply(coor_jgeo,axis=1)
old.to_json('output/old.json',orient="records")

## Importing from API google maps

In [1]:
import requests 
import json
import os 

from dotenv import load_dotenv
load_dotenv()
import sys
sys.path.insert(0, 'src/')
import apiquery # module with the db data

##### --> Cities to import 

We are going to limit the search to specific cities:

* Madrid (40.416977, -3.703827)
* New York (40.7142715 ,-74.0059662)
* Berlin (52.5243683, 13.4105301)
* Seatle (47.6062088, -122.3320694)

In [2]:
apiquery.genera_bd() # <-- Función que consulta el API de google

{'Content-Type': 'application/json; charset=UTF-8', 'Date': 'Sun, 19 Apr 2020 18:33:15 GMT', 'Expires': 'Sun, 19 Apr 2020 18:38:15 GMT', 'Cache-Control': 'public, max-age=300', 'Vary': 'Accept-Language', 'Content-Encoding': 'gzip', 'Server': 'scaffolding on HTTPServer2', 'Content-Length': '4694', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Server-Timing': 'gfet4t7; dur=1027', 'Alt-Svc': 'quic=":443"; ma=2592000; v="46,43",h3-Q050=":443"; ma=2592000,h3-Q049=":443"; ma=2592000,h3-Q048=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,h3-T050=":443"; ma=2592000'}
res:<Response [200]> - item:('starbucks', 'text') - status:OK
https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=CqQCHgEAAIyE-L2IVgK5jKAm0eUwinU8QuWoC0J6dA-PtZvsgqA-y6XBcF8nZ-30V-pQaBjCQCV8Fc3ABsQO5ujkuUjUbwVBWZ3AG8YTctPLOR0Bbxb6uU1ACqP8RxUXisT-hyV19ae-sp1wkvZfNEd82QT2tRMx_q22ZbF0b1XXfTXZsP7V6v-H21_pKLhZ-KtNB3g-TlBot3LmIslidAiUR8-yqXblN9u8R4RWg84E-fQubbsvsOCj5YjjyiC_fMW9XW

{'Content-Type': 'application/json; charset=UTF-8', 'Date': 'Sun, 19 Apr 2020 18:33:42 GMT', 'Expires': 'Sun, 19 Apr 2020 18:38:42 GMT', 'Cache-Control': 'public, max-age=300', 'Vary': 'Accept-Language', 'Content-Encoding': 'gzip', 'Server': 'scaffolding on HTTPServer2', 'Content-Length': '3574', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Server-Timing': 'gfet4t7; dur=697', 'Alt-Svc': 'quic=":443"; ma=2592000; v="46,43",h3-Q050=":443"; ma=2592000,h3-Q049=":443"; ma=2592000,h3-Q048=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,h3-T050=":443"; ma=2592000'}
res:<Response [200]> - item:('kindergarden', 'text') - status:OK
https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken=CrQCIQEAAHNR8cG8gPc0IBCUE0Fc3wBSA23J7m8zZdYTaW3cKnOo5nowPAi5TAUi0c-4rtZ9u5o6kIKYygmOVs8fH_sqgfZeX8e4wrpsPTqCbYvXwK3fcuLkq7SWBrgIMonEyNwan5mPPW98N2NwqQ-lJowovI96JVVMnW_e1Bvc-TB3H6ZTjEgpMOIUCJJTOjaznO6KwjpH0V1PAAPnMuiJoAvFnbmWg2q5CXNiNYTFI8QdPOklbSKyKFd22nQaVE28

True

In [ ]:
# The research is by city:
sys.path.insert(0, 'src/')
import research_mad

point_madrid=(40.416977, -3.703827)

Val_by_place={"other Design companies":val_design_madrid(point_madrid),
         "kindergarden":val_kindergarden_madrid(point_madrid),
         "success startup":val_succ_madrid(point_madrid),
         "Starbucks":val_starbucks_madrid(point_madrid),
         "Airport_or_train_station":val_airport_madrid(point_madrid),
         "place_to party":val_party_madrid(point_madrid),
         "basqueball court":val_basquet_madrid(point_madrid),
         "no_2km_old_company":val_old_madrid(point_madrid),
         "vegan_restaurant":val_vegetariano_madrid(point_madrid)}
